### Resource Allocator and Timeout with Message Passing (Go)

Consider the following resource allocator as an example of an active monitor. It is similar to the one in the course notes.

In [1]:
%%writefile allocator.go
package main

import ("time"; "math/rand")

func printboolarray(a []bool) {
    for _, e := range(a) {
        if e {print(" ")} else {print("X")}
    }
    println()
}
func allocator(capacity int, request chan chan int, release chan int) {
    avail := make([]bool, capacity)
    for i := 0; i < capacity; i++ {avail[i] = true}
    next := 0
    // invariant: 0 <= next && (next < capacity && avail[next] ||
    //              (next == capacity && !avail[0] && ... && !avail[capacity - 1]))
    for {
        if next < capacity { // avail[next]
            select {
            case reply := <- request: {reply <- next; avail[next] = false}
            case unit := <- release: avail[unit] = true
            }
        } else { // !avail[0] && ... && !avail[capacity - 1]
            unit := <- release; avail[unit] = true
        }
        // now comes the computation that takes place between client communication
        printboolarray(avail)
        next = 0; for next < capacity && !avail[next] {next++}
    }
}
func client(i int, request chan chan int, release chan int) {
    reply := make(chan int)
    for {
        request <- reply; unit := <- reply
        time.Sleep(time.Second * time.Duration(rand.Int() % 9)) // sleep between 0 and 8 sec
        release <- unit
    }
}
func main() {
    request, release := make(chan chan int), make(chan int)
    go allocator(5, request, release) // 5 resources
    for i := 0; i < 10; i++ {go client(i, request, release)} // 10 clients
    time.Sleep(time.Second * 6)
}

Writing allocator.go


When clients send a request, they may wait arbitrarily long to receive a reply if the resources are in heavy demand, which may not be desirable. Modify the resource allocator to have timeouts: if a request can be satisfied within `1 sec`, the allocator sends back the available resource unit and, otherwise, a negative number indicating that no resource was allocated. That is, the timeout of a request is activated when the requesting message is received. While activated, further request and release messages can be received. All further requests should be rejected immediately by sending a negative number back. The first received release message deactivates the timeout, and the just-released unit is sent back to the waiting client.

Modify the above implementation accordingly and document it. In particular, state all the essential loop invariants. You may add print statements to visualize the execution and remove the ones included below. *Hint:* The library function [After](https://golang.org/pkg/time/#After) returns (immediately) a channel over which a message is being sent after the specified duration. You can use that to implement the timeout.

In [ ]:
%%writefile allocator.go
package main

import ("time"; "math/rand")

func printboolarray(a []bool) {
    for _, e := range(a) {
        if e {print(" ")} else {print("X")}
    }
    println()
}
# YOUR CODE HERE
raise NotImplementedError()
func main() {
    const C = 5 // capacity
    const R = 100 // repetions of non-timed tests
    const T = 10 // repetitions of timed tests
    request, release, reply := make(chan chan int), make(chan int), make(chan int)
    
    go allocator(C, request, release)
    
    // testing allocator up to capacity
    var a [C]bool // available resources, initially false
    for i := 0; i < R; i++ {
        u := rand.Int() % C // randomly request or release a resource
        if a[u] {release <- u; a[u] = false
        } else {
            request <- reply; r := <- reply
            if r < 0 {panic("available resource not allocated")
            } else if r >= C {panic("improper resource allocated")
            } else if a[r] {panic("same resource used twice")
            } else {a[r] = true
            }            
        }
    }
    for u := 0; u < C; u++ {if a[u] {release <- u; a[u] = false}}
    
    // testing allocator above capacity but without timeout
    for u := 0; u < C; u++ {request <- reply; a[<- reply] = true} // request all resources
    for u := 0; u < C; u++ {if !a[u] {panic("resources not properly allocated")}}
    for i := 0; i < R; i++ {
        request <- reply // request one more resource; cannot be satisfied immediatley
        release <- 0 // release one resource; above request can now be satisfied
        select { // we give the resource allocator .2 seconds to satisfy the request
        case <- time.After(200 * time.Millisecond): panic("released resouce not taken")
        case <- reply: // ok; now all resources are taken
        }
    }
    
    // testing allocator above capacity with timeout
    for i := 0; i < T; i++ {
        request <- reply // request one more resource; cannot be satisfied immediatley
        time.Sleep(1200 * time.Millisecond) // resource allocator should timeout after 1 second
        select {
        case r := <- reply: if r >= 0 {panic("negative reply expected")}
        default: panic("timeout within 1 second expected")
        }
    }

    // testing allocator with repeated requests above capacity
    request <- reply // request one more resource; cannot be satisfied immediatley
    for j := 0; j < R; j++ {
        reply2 := make(chan int)
        request <- reply2 // request another resources; request should be rejected immediately
        select { // we still give it .2 seconds to be rejected
        case r := <- reply2: if r >= 0 {panic("negative reply expected")}
        case <-time.After(200 * time.Millisecond): panic("immediate rejection of second requests expected")
        }
    }
}

In [2]:
!go run allocator.go

X    
XX   
XXX  
XXXX 
XXXXX
XXXX 
XXXXX
XXX X
X X X
XXX X
XXXXX
X XXX
XXXXX
XXXX 
XXXXX


The above implementation activates a timeout only for the first request that cannot be satisfied immediately and rejects all further requests immediately. Modify it such that there is a timeout of `1 sec` for all requests that cannot be satisfied immediately. That is, each request times out `1 sec` after it has been received, so different requests will time out at different times, and the resource allocator has to keep track of all those. Request messages are to be served on a first-come, first-serve basis. Document your design decisions! In particular, state all the essential loop invariants. For implementing a queue, you may use the Go library, e.g. [container/list](https://golang.org/pkg/container/list/). [2 bonus points]

In [ ]:
%%writefile allocator.go
package main

import ("time"; "math/rand"; "container/list")

func printboolarray(a []bool) {
    for _, e := range(a) {
        if e {print(" ")} else {print("X")}
    }
    println()
}
# YOUR CODE HERE
raise NotImplementedError()
func main() {
    const C = 5 // capacity
    const R = 100 // repetitons of non-timed tests
    const T = 10 // repetitons of timed tests
    request, release, reply := make(chan chan int), make(chan int), make(chan int)
    
    go allocator(C, request, release)
    
    // testing allocator up to capacity
    var t [C]bool // taken resources, initially false
    for i := 0; i < R; i++ {
        u := rand.Int() % C // randomly request or release a resource
        if t[u] {release <- u; t[u] = false
        } else {
            request <- reply; r := <- reply
            if r < 0 {println(i, r); panic("available resource not allocated")
            } else if r >= C {panic("improper resource allocated")
            } else if t[r] {panic("same resource used twice")
            } else {t[r] = true
            }            
        }
    }
    for u := 0; u < C; u++ {if t[u] {release <- u; t[u] = false}}

    // testing allocator above capacity but without timeout
    for u := 0; u < C; u++ {request <- reply; t[<- reply] = true} // request all resources
    for u := 0; u < C; u++ {if !t[u] {panic("resources not properly allocated")}}
    for i := 0; i < R; i++ {
        reply2 := make(chan int)
        request <- reply // request one more resource; cannot be satisfied immediatley
        request <- reply2 // yet another request that cannot be satisfied immediately
        release <- 0
        select { // we give the resource allocator .2 seconds to satisfy the first request
        case <- time.After(200 * time.Millisecond): panic("first released resouce not taken")
        case <- reply: // first request satisfied
            release <- 1 // release two resources; above requests can now be satisfied
            select {// we give the resource allocator .2 seconds to satisfy the second request
            case <- time.After(200 * time.Millisecond): panic("second released resouce not taken")
            case <- reply2: // ok                
            }
        }
    }

    // testing allocator above capacity with timeout
    for i := 0; i < T; i++ {
        request <- reply // request one more resource; cannot be satisfied immediatley
        time.Sleep(1200 * time.Millisecond) // resource allocator should timeout after 1 second
        select {// we give the resource allocator .2 seconds to reply
        case <- time.After(200 * time.Millisecond): panic("rejection after 1 second expected")
        case r := <- reply: if r >= 0 {panic("negative reply expected")}
        }
    }

    // testing allocator with repeated requests above capacity
    for i := 0; i < T; i++ {
        request <- reply // request one more resource; cannot be satisfied immediatley
        reply2 := make(chan int)
        request <- reply2 // request another resource
        time.Sleep(1200 * time.Millisecond) // resource allocator should timeout after 1 second
        select { // we still give it .2 seconds to reject the first request
        case <- time.After(200 * time.Millisecond): panic("rejection of first requests expected")
        case r := <- reply:
            if r >= 0 {panic("negative reply expected")}
            select { // we give it another .2 seconds to reject the second request
            case <- time.After(200 * time.Millisecond): panic("rejection of second requests expected")
            case r := <- reply2: if r >= 0 {panic("negative reply expected")}
            }
        }
    }
}

In [ ]:
!go run allocator.go